# This is a webCrawl project
# Gathering craigsList Tech Employement Data.

In [44]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd

In [45]:
url = "https://newyork.craigslist.org/search/npo"

In [46]:
npo_jobs = {}
job_no = 0
while True:
    
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data,'html.parser')
    jobs = soup.find_all('p',{'class':'result-info'})
    
    for job in jobs:
        
        title = job.find('a',{'class':'result-title'}).text
        location_tag = job.find('span',{'class':'result-hood'})
        location = location_tag.text[2:-1] if location_tag else "N/A"
        date = job.find('time', {'class': 'result-date'}).text
        link = job.find('a', {'class': 'result-title'}).get('href')
        
        job_response = requests.get(link)
        job_data = job_response.text
        job_soup = BeautifulSoup(job_data, 'html.parser')
        job_description = job_soup.find('section',{'id':'postingbody'}).text
        job_attributes_tag = job_soup.find('p',{'class':'attrgroup'})
        job_attributes = job_attributes_tag.text if job_attributes_tag else "N/A"
        
        job_no+=1
        npo_jobs[job_no] = [title, location, date, link, job_attributes, job_description]
        
        
#       print('Job Title:', title, '\nLocation:', location, '\nDate:', date, '\nLink:', link,"\n", job_attributes, '\nJob Description:', job_description,'\n---')
        
    url_tag = soup.find('a',{'title':'next page'})
    if url_tag.get('href'):
        url= 'https://newyork.craigslist.org' + url_tag.get('href')
        print(url)
    else:
        break
        
print("Total Jobs:", job_no)
npo_jobs_df = pd.DataFrame.from_dict(npo_jobs, orient = 'index', columns = ['Job Title','Location','Date', 'Link', 'Job Attributes', 'Job Description'])

Total Jobs: 120


In [47]:
npo_jobs_df.head()

,Job Title,Location,Date,Link,Job Attributes,Job Description
1,Program Facilitator,"New York, NY",Jan 5,https://newyork.craigslist.org/mnh/npo/d/new-y...,\ncompensation: $65-$100 per day\n\nemployment...,\n\nQR Code Link to This Post\n\n\nAbout Front...
2,"JOB PLACEMENT; PAID INTERNSHIP, TRAINING $$ AN...",ALL 5 BORO'S,Jan 4,https://newyork.craigslist.org/brx/npo/d/bronx...,"\ncompensation: Paid Internship $1,500 and Emp...",\n\nQR Code Link to This Post\n\n\nECKERD CONN...
3,"CAMPAIGN JOBS! $17-$19/HOUR, OT, BONUSES! HIRI...",Brooklyn,Jan 4,https://newyork.craigslist.org/brk/npo/d/brook...,\ncompensation: $17-$19 per hour\n\nemployment...,\n\nQR Code Link to This Post\n\n\n\nCall Camp...
4,"CAMPAIGN JOBS! $17-$19/HOUR, OT, BONUSES! HIRI...",Flushing,Jan 4,https://newyork.craigslist.org/que/npo/d/flush...,\ncompensation: $17-$19 per hour\n\nemployment...,\n\nQR Code Link to This Post\n\n\n\nCall Camp...
5,Join the Fight Against Poverty(BKnpo),N/A,Jan 3,https://newyork.craigslist.org/brk/npo/d/new-y...,\ncompensation: 15-22/hr*\n\nemployment type: ...,\n\nQR Code Link to This Post\n\n\nUp-Fundrais...


In [48]:
npo_jobs_df.to_csv('npo_jobs.csv')